# Vacantes por institución — Región de Los Ríos

Este cuaderno genera únicamente la visualización de vacantes totales por universidad/IES solicitada para exportarla a HTML.

In [ ]:
from pathlib import Path
import unicodedata

import pandas as pd
import plotly.express as px

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / 'data').exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

data_path = PROJECT_ROOT / 'data/processed/oferta_formativa_clean.parquet'
if not data_path.exists():
    raise FileNotFoundError(f'No se encontró el archivo: {data_path}')

df = pd.read_parquet(data_path)

def normalize(value: str) -> str:
    if not isinstance(value, str):
        return ''
    normalized = unicodedata.normalize('NFKD', value)
    return ''.join(ch for ch in normalized if not unicodedata.combining(ch)).lower()

mask = df['region_sede'].apply(normalize).str.contains('los rios', na=False)
los_rios = df.loc[mask].copy()
if los_rios.empty:
    raise ValueError('No hay registros para la Región de Los Ríos')

vacantes_ies = (
    los_rios.groupby('nombre_ies', as_index=False)
    .agg(vacantes_totales=('total_vacantes', 'sum'))
    .sort_values('vacantes_totales', ascending=True)
)

fig_ies = px.bar(
    vacantes_ies,
    x='vacantes_totales',
    y='nombre_ies',
    orientation='h',
    text='vacantes_totales',
    title='Vacantes totales por institución (Región de Los Ríos)',
    labels={'nombre_ies': 'Institución', 'vacantes_totales': 'Vacantes'}
)
fig_ies.update_layout(height=600)
fig_ies